In [12]:
# Install required libraries
!pip install transformers datasets torch --quiet

# Step 1: Load IMDb dataset
from datasets import load_dataset
dataset = load_dataset('imdb')
train_dataset = dataset['train'].shuffle(seed=42).select(range(2000))  # small subset for fast training
test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

# Step 2: Tokenize using DistilBERT
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Step 3: Load pre-trained model for classification
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=2
)

# Step 4: Define training arguments (minimal for compatibility)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

# Step 5: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

# Step 6: Train model
trainer.train()

# Step 7: Evaluate
results = trainer.evaluate()
print(results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hassaan-solves (hassaan-solves-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


{'eval_loss': 0.2699139416217804, 'eval_runtime': 14.2186, 'eval_samples_per_second': 70.33, 'eval_steps_per_second': 8.791, 'epoch': 1.0}


In [13]:
model.save_pretrained("/content/sentiment_model")
tokenizer.save_pretrained("/content/sentiment_model")
!ls /content/sentiment_model


config.json	   special_tokens_map.json  vocab.txt
model.safetensors  tokenizer_config.json


In [14]:
!zip -r sentiment_model.zip /content/sentiment_model


updating: content/sentiment_model/ (stored 0%)
updating: content/sentiment_model/config.json (deflated 45%)
updating: content/sentiment_model/tokenizer_config.json (deflated 75%)
updating: content/sentiment_model/model.safetensors (deflated 8%)
updating: content/sentiment_model/vocab.txt (deflated 53%)
updating: content/sentiment_model/special_tokens_map.json (deflated 42%)


In [15]:
from google.colab import files
files.download("sentiment_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load the saved model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("/content/sentiment_model")
tokenizer = DistilBertTokenizer.from_pretrained("/content/sentiment_model")

# Predict function
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][prediction].item()
    return {"prediction": prediction, "confidence": confidence}

# Example predictions
print(predict_sentiment("I absolutely loved this movie!"))
print(predict_sentiment("It was boring and disappointing."))


{'prediction': 1, 'confidence': 0.9720109701156616}
{'prediction': 0, 'confidence': 0.9745967984199524}


In [17]:
!pip install gradio --quiet

import gradio as gr
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("/content/sentiment_model")
tokenizer = DistilBertTokenizer.from_pretrained("/content/sentiment_model")

# Prediction function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = "Positive" if torch.argmax(probs) == 1 else "Negative"
        confidence = probs[0][torch.argmax(probs)].item()
    return f"{prediction} ({confidence:.2f} confidence)"

# Gradio UI
gr.Interface(fn=predict, inputs="text", outputs="text", title="Sentiment Classifier").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f99fed3b8b7ca81a99.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
